In [1]:
!pip install datasets transformers[sentencepiece]
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import Dataset, DatasetDict

     |████████████████████████████████| 325 kB 5.6 MB/s 
     |████████████████████████████████| 3.8 MB 16.4 MB/s 
     |████████████████████████████████| 1.1 MB 35.0 MB/s 
     |████████████████████████████████| 134 kB 40.2 MB/s 
     |████████████████████████████████| 212 kB 37.3 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 127 kB 31.9 MB/s 
     |████████████████████████████████| 144 kB 42.2 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
     |████████████████████████████████| 271 kB 43.3 MB/s 
     |████████████████████████████████| 6.5 MB 39.1 MB/s 
     |████████████████████████████████| 895 kB 17.2 MB/s 
     |████████████████████████████████| 596 kB 49.1 MB/s 
     |████████████████████████████████| 1.2 MB 39.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
   

In [2]:
path_to_text = 'books_and_authors'
data = pd.read_csv(path_to_text, names=['v1', 'v2'])


In [3]:
from sklearn.model_selection import train_test_split

trainX, testX, trainY, testY = train_test_split(
    data['v2'], data['v1'], test_size=0.3, random_state=42)


In [4]:
#encode lables (0-ham, 1-spam)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
trainY= le.fit_transform(trainY)
testY = le.fit_transform(testY)
trainY.shape,testY.shape

((699,), (300,))

In [5]:
def prepare_data(X, y):
    data = []
    for i, (sequence, label) in enumerate(zip(X, y)):
        data.append(
        {"paragraph":sequence,
        "label":label,
        "idx":i
        })
        df = pd.DataFrame(data)
    return Dataset.from_pandas(df)

In [6]:
authors_dataset = DatasetDict()

authors_dataset["train"] = prepare_data(trainX, trainY)
authors_dataset["validation"] = prepare_data(testX, testY)

In [7]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["paragraph"], truncation=True)

tokenized_datasets = authors_dataset.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
columns=["attention_mask", "input_ids", "token_type_ids"],
label_cols=["labels"],
shuffle=True,
collate_fn=data_collator,
batch_size=16,
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
columns=["attention_mask", "input_ids", "token_type_ids"],
label_cols=["labels"],
shuffle=False,
collate_fn=data_collator,
batch_size=16,
)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [1]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=7)
from tensorflow.keras.optimizers.schedules import PolynomialDecay



NameError: name 'checkpoint' is not defined

In [12]:
batch_size = 16
num_epochs = 5
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)

In [13]:
import tensorflow as tf

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=7)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])
model_history = model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=num_epochs)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_layer_norm', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_79']
You should probably TRAIN this model on a down-stream task to be able to use i

Epoch 1/5
43/43 [==============================] - 93s 2s/step - loss: 1.7311 - accuracy: 0.3401 - val_loss: 1.3743 - val_accuracy: 0.5933
Epoch 2/5
43/43 [==============================] - 74s 2s/step - loss: 0.9286 - accuracy: 0.7878 - val_loss: 0.6743 - val_accuracy: 0.8533
Epoch 3/5
43/43 [==============================] - 76s 2s/step - loss: 0.3679 - accuracy: 0.9506 - val_loss: 0.4319 - val_accuracy: 0.8800
Epoch 4/5
43/43 [==============================] - 74s 2s/step - loss: 0.1628 - accuracy: 0.9942 - val_loss: 0.3637 - val_accuracy: 0.8900
Epoch 5/5
43/43 [==============================] - 76s 2s/step - loss: 0.1133 - accuracy: 0.9956 - val_loss: 0.3691 - val_accuracy: 0.9033
